In [ ]:
# This information helps with debugging and getting support :)
import sys, platform
import pandas as pd
import bifacial_radiance as br
print("Working on a ", platform.system(), platform.release())
print("Python version ", sys.version)
print("Pandas version ", pd.__version__)
print("bifacial_radiance version ", br.__version__)

# 13 - Modeling Modules with Glass


This journal shows how to add glass to the modules. It also shows how to sample irradiance on the surface of the glass, as well as on the surface of the module. Surface of the module is slightlyt less irradiance due to fresnel losses (a.k.a. Angle of Incidence losses (AOI))

Some examples of the module with glass we'll create:

![Bottom Reflection](../images_wiki/Journal13Pics/Glass_bottom_reflection.PNG)
![Side closeup](../images_wiki/Journal13Pics/Glass_side_closeup.PNG)
![Tiled view](../images_wiki/Journal13Pics/Glass_tilted_reflection.PNG)


In [1]:
import os
from bifacial_radiance import *
from pathlib import Path

testfolder = str(Path().resolve().parent.parent / 'bifacial_radiance' / 'TEMP' / 'Tutorial_13')
if not os.path.exists(testfolder):
    os.makedirs(testfolder)

print ("Your simulation will be stored in %s" % testfolder)


Your simulation will be stored in C:\Users\mprillim\sam_dev\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_13


In [2]:
demo = RadianceObj('tutorial_13',str(testfolder))  
demo.setGround(0.30)  # This prints available materials.
epwfile = demo.getEPW(lat = 37.5, lon = -77.6)  # This location corresponds to Richmond, VA.
metdata = demo.readWeatherFile(epwfile) 
demo.gendaylit(8)  # January 1 4pm (timepoint # 8)\


path = C:\Users\mprillim\sam_dev\bifacial_radiance\bifacial_radiance\TEMP\Tutorial_13
Loading albedo, 1 value(s), 0.300 avg
1 nonzero albedo values.
Getting weather file: USA_VA_Richmond.724010_TMY2.epw
 ... OK!
8760 line in WeatherFile. Assuming this is a standard hourly WeatherFile for the year for purposes of saving Gencumulativesky temporary weather files in EPW folder.
Coercing year to 2021
Saving file EPWs\metdata_temp.csv, # points: 8760
Calculating Sun position for Metdata that is right-labeled  with a delta of -30 mins. i.e. 12 is 11:30 sunpos


'skies\\sky2_37.5_-77.33_2021-01-01_1600.rad'

## Modeling example with glass

In [3]:
module_type = 'test-module' 

numcellsx = 6
numcellsy = 12
xcell = 0.156
ycell = 0.156
xcellgap = 0.02
ycellgap = 0.02

visible = True
diameter = 0.15
tubetype = 'round'
material = 'Metal_Grey'
xgap = 0.1
ygap = 0
zgap = 0.05
numpanels = 1
axisofrotationTorqueTube = False
glass = True

cellLevelModuleParams = {'numcellsx': numcellsx, 'numcellsy':numcellsy, 
                         'xcell': xcell, 'ycell': ycell, 'xcellgap': xcellgap, 'ycellgap': ycellgap}

mymodule = demo.makeModule(name=module_type, x=1, y=1, xgap=xgap, ygap=ygap, 
                           zgap=zgap, numpanels=numpanels, glass=glass, z=0.0002)
mymodule.addTorquetube(material=material, tubetype=tubetype, diameter=diameter,
                      axisofrotation=axisofrotationTorqueTube, recompile=False)
mymodule.addCellModule(**cellLevelModuleParams)

sceneDict = {'tilt':0,'pitch':5.5,'hub_height':1.0,'azimuth':90, 'nMods': 20, 'nRows': 1, 'originx':0, 'originy':0} 
scene = demo.makeScene(module_type, sceneDict)
octfile = demo.makeOct(demo.getfilelist())  


Module Name: test-module


Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten



Module was shifted by 0.078 in X to avoid sensors on air
This is a Cell-Level detailed module with Packaging Factor of 0.81 %
Module test-module updated in module.json
Pre-existing .rad file objects\test-module.rad will be overwritten

Created tutorial_13.oct


## Advanced Rendering:

The images at the beginning of the journal can be made pretty with advanced rendering. This is the workflow:

    rvu -> rpict -> pcond -> pfilt -> ra_tiff -> convert

In detail:

***1. Use rvu to view the oct file***

    rvu 1axis_07_01_08.oct 

Use aim and origin to move around, zoom in/out, etc. Save a view file with view render.

***2. Run rpict to render the image to hdr***

This is the time consuming step. It takes between 1 and 3 hours depending on how complex the geometry is.

    rpict -x 4800 -y 4800 -i -ps 1 -dp 530 -ar 964 -ds 0.016 -dj 1 -dt 0.03 -dc 0.9 -dr 5 -st 0.12 -ab 5 -aa 0.11 -ad 5800 -as 5800 -av 25 25 25 -lr 14 -lw 0.0002 -vf render.vf bifacial_example.oct > render.hdr


***3. Run pcond to mimic human visual response***

    pcond -h render.hdr > render.pcond.hdr


***4. Resize and adjust exposure with pfilt***

    pfilt -e +0.2 -x /4 -y /4 render.pcond.hdr > render.pcond.pfilt.hdr


***5. Convert hdr to tif***

    ra_tiff render.pcond.pfilt.hdr render.tif


***6. Convert tif to png with imagemagick convert utility***

    convert render.tif render.png


***7. Annotate the image with convert***

    convert render.png -fill black -gravity South -annotate +0+5 'Created with NREL bifacial_radiance https://github.com/NREL/bifacial_radiance' render_annotated.png


In [4]:
analysis = AnalysisObj(octfile, demo.basename)

## Scanning Outside of the module, the surface of the glass

In [5]:
frontscan, backscan = analysis.moduleAnalysis(scene)
results = analysis.analysis(octfile, demo.basename, frontscan, backscan)  
load.read1Result('results\irr_tutorial_13.csv')

Linescan in process: tutorial_13_Front
Linescan in process: tutorial_13_Back
Saved: results\irr_tutorial_13.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.8368,5.123922e-17,1.0012,0.999,a9.0.a2.1.0.cellPVmodule.6457,a9.0.a2.1.0.cellPVmodule.2310,70.839097,13.403263,0.189204
1,0.6276,3.842942e-17,1.0012,0.999,a9.0.a2.2.0.cellPVmodule.6457,a9.0.a2.2.0.cellPVmodule.2310,70.839807,12.617947,0.178117
2,0.4184,2.561961e-17,1.0012,0.999,a9.0.a2.3.0.cellPVmodule.6457,a9.0.a2.3.0.cellPVmodule.2310,70.840520,11.433667,0.161398
3,0.2092,1.280981e-17,1.0012,0.999,a9.0.a2.4.0.cellPVmodule.6457,a9.0.a2.4.0.cellPVmodule.2310,70.841237,11.088730,0.156527
4,0.0000,6.162976e-33,1.0012,0.999,a9.0.a0.test-module_Glass.2310,a9.0.a0.test-module_Glass.6457,70.710090,28.340940,0.400799
5,-0.2092,-1.280981e-17,1.0012,0.999,a9.0.a2.7.0.cellPVmodule.6457,a9.0.a2.7.0.cellPVmodule.2310,70.842670,11.450713,0.161634
6,-0.4184,-2.561961e-17,1.0012,0.999,a9.0.a2.8.0.cellPVmodule.6457,a9.0.a2.8.0.cellPVmodule.2310,70.843380,11.866143,0.167496
7,-0.6276,-3.842942e-17,1.0012,0.999,a9.0.a2.9.0.cellPVmodule.6457,a9.0.a2.9.0.cellPVmodule.2310,70.844097,12.713710,0.179458
8,-0.8368,-5.123922e-17,1.0012,0.999,a9.0.a2.10.0.cellPVmodule.6457,a9.0.a2.10.0.cellPVmodule.2310,70.844817,13.386853,0.188958


## Scanning Inside of the module, the surface of the cells

In [6]:
frontscan, backscan = analysis.moduleAnalysis(scene, frontsurfaceoffset=0.05, backsurfaceoffset = 0.05)
results = analysis.analysis(octfile, demo.basename, frontscan, backscan)  
load.read1Result('results\irr_tutorial_13.csv')

Linescan in process: tutorial_13_Front
Linescan in process: tutorial_13_Back
Saved: results\irr_tutorial_13.csv


,x,y,z,rearZ,mattype,rearMat,Wm2Front,Wm2Back,Back/FrontRatio
0,0.8368,5.123922e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905113,13.428140,0.189379
1,0.6276,3.842942e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905143,12.691430,0.178989
2,0.4184,2.561961e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905173,11.462650,0.161659
3,0.2092,1.280981e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905200,11.197587,0.157921
4,0.0000,6.162976e-33,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.773373,28.340940,0.400441
5,-0.2092,-1.280981e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905263,11.457723,0.161590
6,-0.4184,-2.561961e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905293,11.981123,0.168971
7,-0.6276,-3.842942e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905323,12.758743,0.179938
8,-0.8368,-5.123922e-17,1.0502,0.95,a9.0.a0.test-module_Glass.6457,a9.0.a0.test-module_Glass.2310,70.905350,13.492713,0.190289
